# Hidden Markov Model for Regime Detection

This notebook implements a Gaussian Hidden Markov Model (HMM) to identify market regimes (bull and bear markets) using weekly SPY data.

In [ ]:
import pandas as pd
import numpy as np
from hmmlearn.hmm import GaussianHMM
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings("ignore")

## Load and Preprocess Data

1.  Load the daily SPY and GLD price data.
2.  Set the 'Date' column as the index.
3.  Remove rows with missing 'SPY' values.
4.  Resample the data to a weekly frequency, using the last price of each week.
5.  Calculate the weekly percentage change in prices to get the returns.

In [ ]:
df = pd.read_csv('data/spy_gld.csv', index_col='Date', parse_dates=True)

# Drop rows with missing SPY data
df.dropna(subset=['SPY'], inplace=True)

# Resample to weekly data, using Friday's data
df_weekly = df.asfreq('W-FRI').ffill()

# Calculate weekly returns
df_returns = df_weekly.pct_change().dropna()

df_returns.head()

## Implement Gaussian HMM

1.  Define a `GaussianHMM` with two components (states) representing bull and bear markets.
2.  Use a rolling window of 104 weeks (2 years) to train the model.
3.  For each window, fit the HMM to the weekly SPY returns.
4.  Predict the hidden state for the last week in the window.
5.  Identify the bull state as the one with the higher mean return.

In [ ]:
# Define the HMM model
model = GaussianHMM(n_components=2, covariance_type="diag", n_iter=1000, random_state=42)

# Define the window size for the rolling analysis
window_size = 104
predicted_states = []

# Fit the model on a rolling basis
for i in range(window_size, len(df_returns)):
    window = df_returns['SPY'].values[i-window_size:i].reshape(-1, 1)
    model.fit(window)
    state = model.predict(window)[-1]
    
    # Identify bull and bear states based on the mean returns
    # The state with the higher mean is considered the 'bull' state.
    if model.means_[0][0] > model.means_[1][0]:
        bull_state = 0
    else:
        bull_state = 1
        
    # Store 0 for bull and 1 for bear
    predicted_states.append(0 if state == bull_state else 1)

# Add the predicted states to the DataFrame
df_returns['state'] = np.nan
df_returns['state'].iloc[window_size:] = predicted_states

## Visualize the Results

Plot the SPY price and color the background to indicate the detected market regime:
-   **Green**: Bull market
-   **Red**: Bear market

In [ ]:
fig, ax = plt.subplots(figsize=(15, 8))

# Plot the SPY price
ax.plot(df_weekly.index, df_weekly['SPY'], color='black', label='SPY Price')
ax.set_ylabel('SPY Price')
ax.set_title('SPY Price with HMM Regimes')
ax.grid(True)

# Color the background based on the predicted state
ax.fill_between(df_returns.index, ax.get_ylim()[0], ax.get_ylim()[1], where=df_returns['state']==0, color='green', alpha=0.3, label='Bull Market')
ax.fill_between(df_returns.index, ax.get_ylim()[0], ax.get_ylim()[1], where=df_returns['state']==1, color='red', alpha=0.3, label='Bear Market')

ax.legend()
plt.show()